In [1]:
from src import API

API.DataAPI.update()
API.DataAPI.prepare_train_data()
API.Predictor.update_factors()

Update Files
Thu Jun 20 00:54:07 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.03 Secs
Thu Jun 20 00:54:07 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.03 Secs
Thu Jun 20 00:54:07 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.02 Secs
Thu Jun 20 00:54:09 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.38 Secs
Thu Jun 20 00:54:14 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.36 Secs
Thu Jun 20 00:54:15 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240618.feather Done! Cost 0.13 Secs
Thu Jun 20 00:54:15 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240619.feather Done! Cost 0.05 Secs
Thu Jun 20 00:54:15 2024 : Updated ~ data\DataBase\DB_models\risk_cov\2024\risk_cov.20240619.feather Done! Cost 0.03 Secs
Thu Jun 20 00:54:15 2024 : Updated ~ data\DataBase\DB_models\risk_spec\2024\risk_spec.20240619.feath

In [2]:
from src import API
API.Predictor.update_factors()

Load  2 DataBlocks...... finished! Cost 0.11 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.15 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new


In [2]:
from src import API
API.ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
